In [2]:
import pandas as pd

df = pd.read_csv('100_Unique_QA_Dataset.csv')

df.head()

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


In [3]:
#tokenize:

def tokenize(text):
    text = text.lower()
    text = text.replace('?', '')
    text = text.replace("'","")
    return text.split()

                        

In [4]:
tokenize('hey u know What')

['hey', 'u', 'know', 'what']

In [5]:
vocab = {'<UNK>':0}


In [15]:
def build_vocab(row):
    tokenized_qn = tokenize(row['question'])
    tokenized_ans = tokenize(row['answer'])
    
    merged_tokens = tokenized_qn + tokenized_ans
    
    for token in merged_tokens:

         if token not in vocab:
        
          vocab[token] = len(vocab)

        
     

In [16]:
df.apply(build_vocab, axis = 1)

0     None
1     None
2     None
3     None
4     None
      ... 
85    None
86    None
87    None
88    None
89    None
Length: 90, dtype: object

In [17]:
len(vocab)

324

In [20]:
def text_to_indices(text, vocab):
    
    indexed_text = []
    
    for token in tokenize(text):
        
        if token in vocab:
            indexed_text.append(vocab[token])
        else:
            indexed_text.append(vocab['<UNK>'])
        
    return indexed_text

In [21]:
text_to_indices("What is campusx", vocab)

[1, 2, 0]

In [22]:
import torch
from torch.utils.data import Dataset, DataLoader

In [42]:
class QADataset(Dataset):
    
    def __init__(self, df, vocab):
        self.df = df
        self.vocab = vocab
    
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, index):
        
        numerical_question = text_to_indices(self.df.iloc[index]['question'], self.vocab)
        numerical_answer = text_to_indices(self.df.iloc[index]['answer'], self.vocab)
        
        return torch.tensor(numerical_question), torch.tensor(numerical_answer)

In [43]:
dataset = QADataset(df, vocab)

In [44]:
dataloader = DataLoader(dataset, batch_size = 1, shuffle = True)

In [45]:
import torch.nn as nn

In [52]:
class SimpleRNN(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim = 50)
        self.rnn = nn.RNN(50, 64, batch_first = True)
        self.fc = nn.Linear(64, vocab_size)
        
    def forward(self, question):
        embedded_question = self.embedding(question)
        hidden, final = self.rnn(embedded_question)
        output = self.fc(final.squeeze(0))
        
        return output
                        

In [53]:
x = nn.Embedding(324, embedding_dim=50)
y = nn.RNN(50, 64, batch_first=True)
z = nn.Linear(64, 324)

a = dataset[0][0].reshape(1,6)
print("shape of a:", a.shape)
b = x(a)
print("shape of b:", b.shape)
c, d = y(b)
print("shape of c:", c.shape)
print("shape of d:", d.shape)

e = z(d.squeeze(0))

print("shape of e:", e.shape)

shape of a: torch.Size([1, 6])
shape of b: torch.Size([1, 6, 50])
shape of c: torch.Size([1, 6, 64])
shape of d: torch.Size([1, 1, 64])
shape of e: torch.Size([1, 324])


In [54]:
learning_rate = 0.001
epochs = 20

In [55]:
model = SimpleRNN(len(vocab))

In [58]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [60]:
for epoch in range(epochs):
    
    total_loss = 0
    
    for question, answer in dataloader:
        
        optimizer.zero_grad()
        
        output = model(question)
        
        loss = criterion(output, answer[0])
        
        loss.backward()
        
        optimizer.step()
        
        total_loss = total_loss + loss.item()
    print(f"Epoch:{epoch+1}, Loss: {total_loss:4f}")

Epoch:1, Loss: 525.484010
Epoch:2, Loss: 457.870992
Epoch:3, Loss: 376.786848
Epoch:4, Loss: 316.947606
Epoch:5, Loss: 266.202223
Epoch:6, Loss: 218.260841
Epoch:7, Loss: 174.514441
Epoch:8, Loss: 136.921537
Epoch:9, Loss: 105.078026
Epoch:10, Loss: 79.982562
Epoch:11, Loss: 61.243532
Epoch:12, Loss: 47.717728
Epoch:13, Loss: 37.984058
Epoch:14, Loss: 30.893043
Epoch:15, Loss: 25.604659
Epoch:16, Loss: 21.360932
Epoch:17, Loss: 18.348966
Epoch:18, Loss: 15.795616
Epoch:19, Loss: 13.736876
Epoch:20, Loss: 12.067052


In [61]:
def predict(model, question, threshold=0.5):

  # convert question to numbers
  numerical_question = text_to_indices(question, vocab)

  # tensor
  question_tensor = torch.tensor(numerical_question).unsqueeze(0)

  # send to model
  output = model(question_tensor)

  # convert logits to probs
  probs = torch.nn.functional.softmax(output, dim=1)

  # find index of max prob
  value, index = torch.max(probs, dim=1)

  if value < threshold:
    print("I don't know")

  print(list(vocab.keys())[index])

In [62]:
predict(model, "What is the largest planet in our solar system?")

jupiter
